**Notebook content:**
- First model: logistic regression 
- 

In [42]:
%reset -f 

In [43]:
from myDefs.defs import *
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report
from sklearn import metrics
from IPython.display import display

pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 200)
#pd.set_option('display.float_format', lambda x: '%.f' % x)
pd.options.display.precision = 4

#import sys
np.set_printoptions(threshold=sys.maxsize) #- print the full NumPy array

# visualization
import seaborn as sns
import matplotlib.pyplot as plt 
from matplotlib.pyplot import figure
%matplotlib inline


In [44]:
INPUT_PATH = DATA_PATH + 'pre_matrix0.xlsx'
OUTPUT_PATH0 = DATA_PATH + 'matrix0_missing_values.xlsx'
OUTPUT_PATH1 = DATA_PATH + 'matrix0.xlsx'

**Read events file**

In [45]:
df = pd.read_excel(INPUT_PATH, header=[0,1,2,3], index_col=0)

In [46]:
df.shape

(525, 894)

In [47]:
df.head()

type      metadata                                                        \
time       general                                                         
feature        Sex      Age   Urgency Surgical approach Anastomotic Leak   
statistic constant constant  constant          constant         constant   
pid                                                                        
1123813       male     59.0  elective              Open                1   
1135541     female     43.0  elective              Open                0   
3918745     female     57.0  elective      Laparoscopic                0   
4894647        NaN      NaN  elective              Open                0   
6541482     female     65.0    urgent              Open                0   

type      numeric_events                                                 \
time      before_surgery                                                  
feature          Albumin                     Alkaline Phosphatase         
statistic            max    mean median  min                  max  mean   
pid                                                                       
1123813              4.2  4.2000   4.20  4.2                 97.0  97.0   
1135541              NaN     NaN    NaN  NaN                  NaN   NaN   
3918745              3.9  3.9000   3.90  3.9                 90.0  90.0   
4894647              4.3  4.3000   4.30  4.3                 70.0  70.0   
6541482              3.1  1.9429   1.95  1.4                123.0  94.0   

type                                     ...             drugs  \
time                                     ...          all_days   
feature                Amylase           ... Magnesium sulfate   
statistic median   min     max     mean  ...             count   
pid                                      ...                     
1123813     97.0  97.0    63.0  63.0000  ...               4.0   
1135541      NaN   NaN     NaN      NaN  ...               NaN   
3918745     90.0  90.0    53.0  53.0000  ...               NaN   
4894647     70.0  70.0     NaN      NaN  ...               1.0   
6541482    112.5  33.0    68.0  32.6923  ...               7.0   

type                                                                           \
time                                                                            
feature   Magnesium sulfate  Meroken new Meropenem Metronidazole Morphine hcl   
statistic              count       count     count         count        count   
pid                                                                             
1123813                  NaN         1.0      22.0          52.0          3.0   
1135541                  NaN         NaN       NaN          26.0          NaN   
3918745                  NaN         NaN       NaN           NaN          NaN   
4894647                  1.0         1.0       NaN          73.0          NaN   
6541482                  NaN         NaN       NaN          51.0          2.0   

type                                                                       \
time                                                                        
feature   Morphine hcl  Norepinephrine Norepinephrine + dextrose Optalgin   
statistic         count          count                     count    count   
pid                                                                         
1123813             NaN           30.0                       NaN      2.0   
1135541             3.0            NaN                       NaN      NaN   
3918745             NaN            NaN                       NaN      NaN   
4894647             NaN            NaN                       NaN      3.0   
6541482             NaN           20.0                       NaN      NaN   

type                                                                         
time                                                                         
feature   Piperacillin / tazobactam Prednisone Propranolol Vaben Vancomycin  
statistic  

In [48]:
n = df.columns.get_level_values('feature').nunique() - df['metadata'].shape[1]
print("number of features:", n)

number of features: 138


# Process data 


## Remove columns by missing values

In [49]:
n = df.columns.get_level_values('feature').nunique() - df['metadata'].shape[1]
print("Num features before dropping columns:", n)

Num features before dropping columns: 138


In [50]:
npatients = df.shape[0]

In [51]:
#filter "numeric events" columns
v = df[['numeric_events']].isna().sum().gt(npatients*0.50) #gt = greater than
df.drop(v.index[v], axis=1, inplace=True)

In [52]:
#filter drugs columns
v = df[['drugs']].isna().sum().gt(npatients*0.80)
df.drop(v.index[v], axis=1, inplace=True)

In [53]:
n = df.columns.get_level_values('feature').nunique() - df['metadata'].shape[1]
print("Num features after dropping columns:", n)

Num features after dropping columns: 71


In [54]:
if WRITE_FLAG:
    df.to_excel(OUTPUT_PATH0)

## Fill missing data

In [55]:
#fill numeric events with median 
df['numeric_events'] = df['numeric_events'].fillna(df['numeric_events'].median())

In [56]:
#fill missing values in dtugs with 0 
df['drugs'] = df['drugs'].fillna(0)

In [57]:
#fill numeric events with median 
df.iloc[:,1] = df.iloc[:,1].fillna(df.iloc[:,1].median())

In [58]:
df.head()

type      metadata                                                        \
time       general                                                         
feature        Sex      Age   Urgency Surgical approach Anastomotic Leak   
statistic constant constant  constant          constant         constant   
pid                                                                        
1123813       male     59.0  elective              Open                1   
1135541     female     43.0  elective              Open                0   
3918745     female     57.0  elective      Laparoscopic                0   
4894647        NaN     55.0  elective              Open                0   
6541482     female     65.0    urgent              Open                0   

type      numeric_events                                                 \
time      before_surgery                                                  
feature          Albumin                     Alkaline Phosphatase         
statistic            max    mean median  min                  max  mean   
pid                                                                       
1123813              4.2  4.2000   4.20  4.2                 97.0  97.0   
1135541              4.1  4.0000   4.00  3.9                 94.0  91.0   
3918745              3.9  3.9000   3.90  3.9                 90.0  90.0   
4894647              4.3  4.3000   4.30  4.3                 70.0  70.0   
6541482              3.1  1.9429   1.95  1.4                123.0  94.0   

type                                     ...                                \
time                                     ... after_surgery                   
feature                Amylase           ...    saturation                   
statistic median   min     max     mean  ...           max     mean median   
pid                                      ...                                 
1123813     97.0  97.0    63.0  63.0000  ...         100.0  97.1693   98.0   
1135541     91.0  84.0    68.0  60.5000  ...         100.0  97.9277   98.0   
3918745     90.0  90.0    53.0  53.0000  ...         100.0  96.7879   97.0   
4894647     70.0  70.0    68.0  60.5000  ...         100.0  96.7396   98.0   
6541482    112.5  33.0    68.0  32.6923  ...         100.0  97.2320   98.0   

type                                              drugs              \
time                                           all_days               
feature         weight                        Cefazolin Ceftriaxone   
statistic   min     max     mean median   min     count       count   
pid                                                                   
1123813    85.0   108.0  96.9667  105.0  85.3       6.0        13.0   
1135541    94.0    68.0  68.0000   68.0  68.0       0.0        10.0   
3918745    91.0    80.0  78.0000   78.0  76.0       0.0         0.0   
4894647     0.0    85.0  71.1250   74.6   0.0       5.0        12.0   
6541482    80.0    58.0  54.6667   53.0  53.0       4.0        15.0   

type                                                                          
time                                                                          
feature   Fleet enema Furosemide Magnesium sulfate Meroken new Metronidazole  
statistic       count      count             count       count         count  
pid                                                                           
1123813           1.0       16.0               4.0         1.0          52.0  
1135541           0.0        2.0               0.0         0.0          26.0  
3918745           0.0        0.0               0.0         0.0           0.0  
4894647           1.0       38.0               1.0         1.0          73.0  
6541482           0.0      151.0               7.0         0.0          51.0  

[5 rows x 480 columns]

In [59]:
'''#read events
path = DATA_PATH + "parseData2.csv"
df = pd.read_csv(path, sep=',') 
#parse date of surgery
df['eventStartDate'] = pd.to_datetime(df['eventStartDate'], format='%Y-%m-%d')
df['eventEndDate'] = pd.to_datetime(df['eventEndDate'], format='%Y-%m-%d')

#read annonymous file
path = "{}parseAnonymous0.csv".format(DATA_PATH)
anonymous = pd.read_csv(path, sep=',')
#parse date of surgery
anonymous['Date of surgery'] = pd.to_datetime(anonymous['Date of surgery'], format='%Y-%m-%d')'''

'#read events\npath = DATA_PATH + "parseData2.csv"\ndf = pd.read_csv(path, sep=\',\') \n#parse date of surgery\ndf[\'eventStartDate\'] = pd.to_datetime(df[\'eventStartDate\'], format=\'%Y-%m-%d\')\ndf[\'eventEndDate\'] = pd.to_datetime(df[\'eventEndDate\'], format=\'%Y-%m-%d\')\n\n#read annonymous file\npath = "{}parseAnonymous0.csv".format(DATA_PATH)\nanonymous = pd.read_csv(path, sep=\',\')\n#parse date of surgery\nanonymous[\'Date of surgery\'] = pd.to_datetime(anonymous[\'Date of surgery\'], format=\'%Y-%m-%d\')'

## Exploration

In [60]:
counts = y.value_counts().to_frame()
counts.index = ["AL", "non-AL"]
counts.columns = ["counts"]
display(counts)

percent = y.sum()/len(y) 
print("percentage of num Anastomotic leak: {:.2f}%".format(percent*100))

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = 'Non-AL', 'Anastomotic Leak (AL)'
sizes = [len(y) - y.sum(), y.sum()]

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

NameError: name 'y' is not defined

# Logistic Regression 

## Get x and y

In [ ]:
to_drop = [df.columns[0], df.columns[2], df.columns[3], df.columns[4]]
x = df.drop(columns=to_drop)
x.head()

In [ ]:
y = df.iloc[:, 4]
y.name = "labels"

### train - test

In [ ]:
#The stratify parameter makes a split so that the proportion of values in the sample produced 
#will be the same as the proportion of values provided to parameter stratify.

x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size=0.2,
                                                    random_state=0,
                                                    stratify=y)

In [ ]:
traintest = pd.concat([y_train.value_counts(), y_test.value_counts()], axis=1).T
traintest.index = ['train', 'test'] 
traintest.columns = ["non-AL", "AL"]
traintest['total'] = traintest.sum(axis=1)
traintest

## fit

In [ ]:
model = LogisticRegression(solver='lbfgs', max_iter=100000) #liblinear
model.fit(x_train, y_train)

### Performances

In [ ]:
y_pred = model.predict(x_test)
score = model.score(x_test, y_test) 
#accuracy: tp + tn / all
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(score))


In [ ]:
#precision is the ratio tp / (tp + fp)
# recall is the ratio tp / (tp / fn)
print(classification_report(y_test, y_pred))

#### confusion matrix

In [ ]:
#plot confusion matrix
#confusion_matrix = confusion_matrix(y_test, y_pred)

disp = plot_confusion_matrix(model, x_test, y_test, display_labels=["no Leak", "Leak"], normalize='true') #normalize='false'
#normalization: first row is divided by number of no leak, second is divided by number of leak
disp.ax_.set_title("Normalized confusion matrix")

#### ROC curve

In [ ]:
#roc curve
probs = model.predict_proba(x_test)[:,1] #first index - probability that data belongs to class 0, second index - to class 1
fpr, tpr, thresholds = metrics.roc_curve(y_test, probs)

#area under curve
logit_roc_auc = metrics.roc_auc_score(y_test, y_pred)

plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = {:.2f})'.format(logit_roc_auc))
plt.plot([0, 1], [0, 1],'r--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.legend(loc="lower right")
plt.show()

### forward selection

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

k_feautres = 3 

#Sequential Forward Selection
sfs = SFS(model, 
          k_features=k_feautres, 
          forward=True, 
          floating=False, 
          scoring='accuracy',
          cv=4,
          n_jobs=-1)
sfs = sfs.fit(X, y)

print('\nSequential Forward Selection (k=3):')
print(sfs.k_feature_idx_)
print('CV Score:')
print(sfs.k_score_)


In [61]:
import mlxtend

ModuleNotFoundError: No module named 'mlxtend'